In [1]:
import os
print(os.getcwd())
current_path = os.getcwd()
current_path  = os.path.dirname(current_path)
current_path = os.path.dirname(current_path)
parent_path = os.path.dirname(current_path)
os.chdir(parent_path)
print(os.getcwd())

C:\Users\theal\PycharmProjects\ensembleLegends\src\deepReplication\deepBollinger
C:\Users\theal\PycharmProjects\ensembleLegends


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.metrics import Precision, Recall
from sklearn.metrics import confusion_matrix
from src.config import DEEP_DATA_PATH
from src.deepReplication.dataPreperation.dataCombining import combine_data
from src.deepReplication.dataPreperation.scaling import scaling

def specificity(y_true, y_pred, threshold=0.5):
    y_pred_thresholded = tf.cast(y_pred > threshold, tf.bool)
    y_true = tf.cast(y_true, tf.bool)

    true_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(y_true), tf.logical_not(y_pred_thresholded)), dtype=tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(y_true), y_pred_thresholded), dtype=tf.float32))

    return true_negatives / (true_negatives + false_positives + tf.keras.backend.epsilon())


precison = Precision()

In [5]:
num_prev_prices = 20

running_data_preprocesing = "Bollinger"
identifier = 'test2bollinger'

split_to_do = "34_33_33"

In [7]:
if running_data_preprocesing == "Bollinger":
    combine_data(base_name='deepBollinger', identifier=identifier, num_prev_prices=num_prev_prices, drop_nans=True, splits=split_to_do)
    # combine_data(base_name='deepBollinger', identifier='test1bollinger', num_prev_prices=20, drop_nans=True)
    scaling(base_name='Bollinger', identifier=identifier, num_prev_price=num_prev_prices, splits=split_to_do)

if running_data_preprocesing == "Turtle":
    combine_data(base_name='deepTurtle', strategy='turtle_naive', identifier=identifier, num_prev_prices=num_prev_prices, drop_nans=True)
    scaling(base_name='Turtle', identifier=identifier, num_prev_price=num_prev_prices)

if running_data_preprocesing == "Box":
    combine_data(base_name='deepBox', strategy='box_naive', identifier=identifier, num_prev_prices=num_prev_prices, drop_nans=True)
    scaling(base_name='Box', identifier=identifier, num_prev_price=num_prev_prices)

350546
350546


Scaling: 100%|████████████████████████████████████████████████████████████████| 697687/697687 [14:08<00:00, 822.05it/s]


In [10]:
path = os.path.join(DEEP_DATA_PATH, 'deepBollinger', f'{split_to_do}_scaled_combined_{identifier}_20100104_to_20201231_doctest_numP{num_prev_prices}.csv')
# path = os.path.join(DEEP_DATA_PATH, 'deepTurtle', 'scaled_combined_turtle_naive_20100104_to_20201231_doctest.csv')
# path = os.path.join(DEEP_DATA_PATH, 'deepBox', 'scaled_combined_box_naive_20100104_to_20201231_doctest.csv')

df = pd.read_csv(path, low_memory=False)

In [11]:
trade_counts = df['is_trade'].value_counts()
print(trade_counts)

0    350546
1    347141
Name: is_trade, dtype: int64


In [12]:
# Prepare input and target
X = df[[f'PrevPrice_{i}' for i in range(num_prev_prices + 1)]].values
y = df['is_trade'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

series = pd.Series(y_test)
trade_counts = series.value_counts()
print(trade_counts)

0    70240
1    69298
dtype: int64


In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(X_train.shape[1],)),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', precison, specificity])

In [14]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
17443/17443 [==============================] - 77s 4ms/step - loss: 0.1045 - accuracy: 0.9614 - precision: 0.9493 - specificity: 0.9485 - val_loss: 0.0746 - val_accuracy: 0.9740 - val_precision: 0.9572 - val_specificity: 0.9565
Epoch 2/5
17443/17443 [==============================] - 75s 4ms/step - loss: 0.0767 - accuracy: 0.9715 - precision: 0.9610 - specificity: 0.9606 - val_loss: 0.0722 - val_accuracy: 0.9718 - val_precision: 0.9716 - val_specificity: 0.9721
Epoch 3/5
17436/17443 [============================>.] - ETA: 0s - loss: 0.0691 - accuracy: 0.9742 - precision: 0.9643 - specificity: 0.9640


KeyboardInterrupt



In [9]:
results = model.evaluate(X_test, y_test)

1561/1561 [==============================] - 4s 2ms/step - loss: 0.1423 - accuracy: 0.9428 - precision: 0.9313 - specificity: 0.9305


In [15]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype("int32")
cm = confusion_matrix(y_test, y_pred)
cm

array([[67538,  2702],
       [  460, 68838]], dtype=int64)

In [26]:
# 1 if X_test[0] == 1 or X_test[0] == 0

def naive_model(X_test, y_test):
    preds = []
    for x in X_test:
        if float(x[0]) > .9 or float(x[0] < .1):
            preds.append(1)
        else:
            preds.append(0)

    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()

    # Calculate metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if tp + fp != 0 else 0
    specificity = tn / (tn + fp) if tn + fp != 0 else 0
    recall = tp / (tp + fn) if tp + fn != 0 else 0  # Added recall calculation

    return accuracy, precision, specificity, recall




In [27]:
acc, prec, spec, recall = naive_model(X_test, y_test)

In [28]:
print(acc, prec, spec, recall)

0.806038498473534 0.7281311107029806 0.6417283599088838 0.9725821813039337


In [ ]:
Epoch 1/5
6244/6244 [==============================] - 28s 4ms/step - loss: 0.2143 - accuracy: 0.9113 - precision: 0.8810 - specificity: 0.8728 - val_loss: 0.1593 - val_accuracy: 0.9390 - val_precision: 0.9093 - val_specificity: 0.9047

6244/6244 [==============================] - 27s 4ms/step - loss: 0.1228 - accuracy: 0.9534 - precision: 0.9344 - specificity: 0.9323 - val_loss: 0.1253 - val_accuracy: 0.9538 - val_precision: 0.9314 - val_specificity: 0.9294

0.6597124894886477 0.7761268306554813 0.874950361369232 0.4408415441770312